In [8]:
import pandas as pd
import numpy as np
import json

In [37]:
# location dataframe
location_df = pd.read_csv('SaO_Optilandia_resub_locations.csv')

# links dataframe
links_df = pd.read_csv('SaO_Optilandia_resub_links.csv')

# extract lorry data from json
lorry_data = json.load(open('SaO_Optilandia_resub_depot_lorries.json', 'r'))

# set count to 0
k = 0

# initialise lorry list
lorry = []

# loop -> set i to the respective lorry key
for i in lorry_data.keys():
    # set j to the the number of lorries at key 'i'
    for j in range(0, len(lorry_data[i])):
        # append each lorry in lorry_data to lorry list
        lorry.append(pd.DataFrame(lorry_data[i][j], index=[k]))
        # accumulate index
        k += 1

# lorry dataframe
lorry_df = pd.concat(lorry)

In [40]:
# show first 5 rows of lorry_df
lorry_df.head()

,lorry_id,capacity,cpm,cptm
0,124-0,8,1.1,1.3
1,124-1,8,1.1,1.3
2,124-2,16,1.8,0.8
3,124-3,25,2.3,0.5
4,127-0,8,1.1,1.3
